In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from ngram import calculate_avg_ll
from plot_utils import plt, legend_opts, savefig

## Combine models sequentially

In [4]:
train_prob_matrix = np.load('../data/train_prob_matrix.npy')
dev1_prob_matrix = np.load('../data/dev1_prob_matrix.npy')
dev2_prob_matrix = np.load('../data/dev2_prob_matrix.npy')

In [5]:
train_avg_lls = []
dev1_avg_lls = []
dev2_avg_lls = []
labels = ['0\n(uniform)']
label = ''

ngram_combo = []
for n in range(6):
    ngram_combo.append(n)
    train_avg_lls.append(calculate_avg_ll(train_prob_matrix, ngram_combo))
    dev1_avg_lls.append(calculate_avg_ll(dev1_prob_matrix, ngram_combo))
    dev2_avg_lls.append(calculate_avg_ll(dev2_prob_matrix, ngram_combo))
    
    label += f'{n}\n'
    labels.append(label)

In [6]:
fig, ax = plt.subplots(figsize=(5, 5))

ax.plot(ngram_combo, train_avg_lls, '-o', color='tab:red', clip_on=False)
ax.plot(ngram_combo, dev1_avg_lls, '-o', color='tab:blue', clip_on=False)
ax.plot(ngram_combo, dev2_avg_lls, '-o', color='tab:green', clip_on=False)

ax.text(5.1, train_avg_lls[-1]+0.1, s='train', va='center', color='tab:red', fontsize=15)
ax.text(5.1, dev1_avg_lls[-1]+0.1, s='dev1', va='center', color='tab:blue', fontsize=15)
ax.text(5.1, dev2_avg_lls[-1]+0.1, s='dev2', va='center', color='tab:green', fontsize=15)

ax.set_xticklabels(labels)
ax.set_xlabel('ngram model')
ax.set_ylabel('Avearge log likelihood')
ax.set_xlim(0, 5)
ax.set_ylim(-15, 0)
ax.set_yticks(range(-15, 1, 3))
plt.show()

NameError: name 'plt' is not defined

## Expectation-maximation algorithm

### 